<a href="https://colab.research.google.com/github/ahakobia/Group4_NFLX_MIDTERM/blob/main/Justin_PySpark_US_Energy_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [948 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:14

In [2]:
# Install pytest and pytest-sugar to make our output look nice.
!pip install -q pytest pytest-sugar

In [3]:
# Create and navigate to the tdd directory.
from pathlib import Path
if Path.cwd().name != 'tests':
    %mkdir tests
    %cd tests
# Show the current working directory.  
%pwd

/content/tests


'/content/tests'

In [4]:
# Initialize the __init__.py file. 
# This file will be stored in our pwd (/content/tests)
%%file __init__.py
pass

Writing __init__.py


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import findspark
findspark.init()

# Import other dependencies. 
from pyspark import SparkFiles
from pyspark.sql import SparkSession

import time

spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
energy = spark.read.csv('../drive/MyDrive/organised_Gen.csv', header = True, inferSchema = True)
energy = energy.withColumnRenamed('TYPE OF PRODUCER', 'producer')
energy = energy.withColumnRenamed('ENERGY SOURCE', 'source')
energy = energy.withColumnRenamed('GENERATION (Megawatthours)', 'generated')
energy.write.parquet('parquet_energy',mode='overwrite')
p_energy = spark.read.parquet('parquet_energy')
p_energy.createOrReplaceTempView('p_energy_data')

start_time = time.time()
transformed_df = spark.sql("""
     SELECT 
            YEAR as year, 
            producer, 
            source, 
            generated AS energy 
        FROM p_energy_data
        WHERE 
            state != 'US-TOTAL' 
            AND producer != 'Total Electric Power Industry' 
            AND source != 'Total'
        """)
transformed_df

  
q_df = spark.sql("""
      SELECT year, producer, SUM(generated) AS total_generated 
                FROM p_energy_data
                WHERE year >= 2001 AND year < 2022 
                    AND (source='Wind' OR source='Solar Thermal and Photovoltaic' 
                        OR source='Hydroelectric Conventional' OR source='Geothermal' 
                        OR source='Wood and Wood Derived Fuels' OR source='Other Biomass' 
                        OR source='Pumped Storage')
                    AND (state != 'US-TOTAL' 
                        AND producer != 'Total Electric Power Industry' 
                        AND source != 'Total')
        GROUP BY year, producer
        ORDER BY year ASC, producer DESC;
                """)
q_df


print("--- %s seconds ---" % (time.time() - start_time))

--- 0.08817100524902344 seconds ---


In [28]:
# Create a bank_data.py file and write the function to it. 
# This file will be stored in our pwd (/content/tests).
%%file total_energy.py

# Import findspark() and initialize. 
import findspark
findspark.init()

# Import other dependencies. 
from pyspark import SparkFiles
from pyspark.sql import SparkSession

# Create the import_data function. 
def import_data():
    spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
    energy = spark.read.csv('../drive/MyDrive/organised_Gen.csv', header = True, inferSchema = True)
    energy = energy.withColumnRenamed('TYPE OF PRODUCER', 'producer')
    energy = energy.withColumnRenamed('ENERGY SOURCE', 'source')
    energy = energy.withColumnRenamed('GENERATION (Megawatthours)', 'generated')
    energy.write.parquet('parquet_energy',mode='overwrite')
    p_energy = spark.read.parquet('parquet_energy')
    p_energy.createOrReplaceTempView('p_energy_data')
    return p_energy

def transform_data():
    spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
    transformed_df = spark.sql("""
        SELECT 
            YEAR as year, 
            producer, 
            source, 
            generated AS energy 
        FROM p_energy_data
        WHERE 
            state != 'US-TOTAL' 
            AND producer != 'Total Electric Power Industry' 
            AND source != 'Total'
            """)
    return transformed_df

def query_data():
    spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
    q_df = spark.sql("""
        SELECT year, producer, SUM(generated) AS total_generated 
                FROM p_energy_data
                WHERE year >= 2001 AND year < 2022 
                    AND (source='Wind' OR source='Solar Thermal and Photovoltaic' 
                        OR source='Hydroelectric Conventional' OR source='Geothermal' 
                        OR source='Wood and Wood Derived Fuels' OR source='Other Biomass' 
                        OR source='Pumped Storage')
                    AND (state != 'US-TOTAL' 
                        AND producer != 'Total Electric Power Industry' 
                        AND source != 'Total')
        GROUP BY year, producer
        ORDER BY year ASC, producer DESC;
                  """)
    return q_df

Overwriting total_energy.py


In [29]:
# Create a test_bank_data.py file and write the test functions to it. 
# This file will be stored in our pwd (/content/tests).
%%file test_total_states_energy.py

from total_energy import (import_data, transform_data, query_data)

# Write the tests. 
def test_row_count_before_transform():
  df = import_data()
  assert df.count() == 496774

def test_column_count_before_transform():
  df = import_data()
  assert len(df.columns) == 7

def test_row_count_after_transform():
    df = transform_data()
    assert df.count() != 496774

def test_column_count_after_transform():
    df = transform_data()
    assert len(df.columns) != 7

def test_row_count_query():
    df = query_data()
    assert df.count() == 105

def test_column_count_query():
    df = query_data()
    assert len(df.columns) == 3

Overwriting test_total_states_energy.py


In [26]:
# Run the test_import_data.py file with pytest. 
!python -m pytest test_total_states_energy.py

Test session starts (platform: linux, Python 3.7.14, pytest 3.6.4, pytest-sugar 0.9.5)
rootdir: /content/tests, inifile:
plugins: typeguard-2.7.1, sugar-0.9.5

 test_total_states_energy.py ✓✓✓✓                                 67% ██████▋   

――――――――――――――――――――――――――――― test_row_count_query ―――――――――――――――――――――――――――――

    def test_row_count_query():
        df = query_data()
>       assert df.count() == 105
E       assert 126 == 105
E        +  where 126 = <bound method DataFrame.count of DataFrame[year: int, producer: string, total_generated: double]>()
E        +    where <bound method DataFrame.count of DataFrame[year: int, producer: string, total_generated: double]> = DataFrame[year: int, producer: string, total_generated: double].count

test_total_states_energy.py:23: AssertionError
----------------------------- Captured stderr call -----------------------------
                                                                                

 test_total_states_energy.py ⨯✓   